In [4]:
from enum import Enum, IntEnum
from tkinter import NO

class FunctionalUnit(Enum):
    Integer = 1
    Mult1 = 2
    Mult2 = 3
    Add = 4
    Divide = 5

FloatRegister = IntEnum("Fegister", ['F0','F1','F2','F3','F4','F5','F6','F7','F8'
                                ,'F9','F10','F11','F12','F13','F14','F15',
                                'F16','F17','F18','F19','F20','F21','F22',
                                'F23','F24','F25','F26','F27','F28','F29',
                                'F30','F31'], start = 0)

Register = IntEnum("Register", ['R0','R1','R2','R3','R4','R5','R6','R7','R8'
                                ,'R9','R10','R11','R12','R13','R14','R15',
                                'R16','R17','R18','R19','R20','R21','R22',
                                'R23','R24','R25','R26','R27','R28','R29',
                                'R30','R31'], start = 0)

class FunctionalEntry:
    def __init__(self) -> None:
        self.time = -1
        self.busy: bool = False
        self.op: str = None
        self.dst: FloatRegister | Register = None
        self.s1 : FloatRegister | Register = None
        self.s2 : FloatRegister | Register = None
        self.s1_fu: FunctionalUnit = None
        self.s2_fu: FunctionalUnit = None
        self.s1_ready: bool = False
        self.s2_ready: bool = False
        self.instr: Instruction = None
    
    def clear(self):
        self.time = -1
        self.busy: bool = False
        self.op: str = None
        self.dst: FloatRegister | Register = None
        self.s1 : FloatRegister | Register = None
        self.s2 : FloatRegister | Register = None
        self.s1_fu: FunctionalUnit = None
        self.s2_fu: FunctionalUnit = None
        self.s1_ready: bool = True
        self.s2_ready: bool = True
        self.instr: Instruction = None

class FunctionalUnitStatus:
    def __init__(self) -> None:
        self.table: dict[FunctionalUnit, FunctionalEntry] = {FunctionalUnit.Integer: FunctionalEntry(),    \
                                                      FunctionalUnit.Mult1: FunctionalEntry(),       \
                                                      FunctionalUnit.Mult2: FunctionalEntry(),       \
                                                      FunctionalUnit.Add: FunctionalEntry(),         \
                                                      FunctionalUnit.Divide: FunctionalEntry() }

class RegisterResultStatus:
    def __init__(self) -> None:
        self.table: list[FunctionalUnit] = [None for _ in range(32)]

class Instruction:
    def __init__(self, name) -> None:
        self.name: str = name
        self.delay = 0
    
    def __str__(self) -> str:
        return self.name

class LD(Instruction):
    def __init__(self, dst: FloatRegister, imm: int, addr: Register) -> None:
        super().__init__("L.D")
        self.dst: FloatRegister = dst
        self.imm: int = imm
        self.addr: Register = addr
        self.delay = 1
    
    def __str__(self) -> str:
        return super().__str__() + "\t" + "{}, {}({})".format(self.dst, self.imm, self.addr)

class MULD(Instruction):
    def __init__(self, dst: FloatRegister, src1: FloatRegister, src2: FloatRegister) -> None:
        super().__init__("MUL.D")
        self.dst: FloatRegister = dst
        self.src1: FloatRegister = src1
        self.src2: FloatRegister = src2
        self.delay = 10
    
    def __str__(self) -> str:
        return super().__str__() + "\t" + "{}, {}, {}".format(self.dst, self.src1, self.src2)
    
class ADDD(Instruction):
    def __init__(self, dst: FloatRegister, src1: FloatRegister, src2: FloatRegister) -> None:
        super().__init__("ADD.D")
        self.dst: FloatRegister = dst
        self.src1: FloatRegister = src1
        self.src2: FloatRegister = src2
        self.delay = 2
    
    def __str__(self) -> str:
        return super().__str__() + "\t" + "{}, {}, {}".format(self.dst, self.src1, self.src2)
    
class SUBD(Instruction):
    def __init__(self, dst: FloatRegister, src1: FloatRegister, src2: FloatRegister) -> None:
        super().__init__("SUB.D")
        self.dst: FloatRegister = dst
        self.src1: FloatRegister = src1
        self.src2: FloatRegister = src2
        self.delay = 2
    
    def __str__(self) -> str:
        return super().__str__() + "\t" + "{}, {}, {}".format(self.dst, self.src1, self.src2)
        
class DIVD(Instruction):
    def __init__(self, dst: FloatRegister, src1: FloatRegister, src2: FloatRegister) -> None:
        super().__init__("DIV.D")
        self.dst: FloatRegister = dst
        self.src1: FloatRegister = src1
        self.src2: FloatRegister = src2
        self.delay = 40
    
    def __str__(self) -> str:
        return super().__str__() + "\t" + "{}, {}, {}".format(self.dst, self.src1, self.src2)

class InstructionStatus:
    def __init__(self) -> None:
        self.issue: int = -1
        self.read_op: int = -1
        self.exec_comp: int = -1
        self.wb: int = -1
    
    def is_wb(self, cycle: int) -> bool:
        return self.exec_comp >= cycle and self.wb == -1

    def is_exec(self, cycle: int) -> bool:
        return self.read_op >= cycle and self.exec_comp == -1
    
    def is_read_op(self, cycle: int) -> bool:
        return self.issue >= cycle and self.read_op == -1

instruction_status: dict[Instruction, InstructionStatus] = {}
register_result_status = RegisterResultStatus()
functional_unit_status = FunctionalUnitStatus()


In [5]:
def parser(instrs: list) -> list[Instruction]:
    res = []
    for instr in instrs:
        if instr[0] == "L.D":
            dst = instr[1]
            imm = instr[2]
            addr = instr[3]
            res.append(LD(dst, imm, addr))
        elif instr[0] == "MUL.D":
            dst = instr[1]
            src1 = instr[2]
            src2 = instr[3]
            res.append(MULD(dst, src1, src2))
        elif instr[0] == "ADD.D":
            dst = instr[1]
            src1 = instr[2]
            src2 = instr[3]
            res.append(ADDD(dst, src1, src2))
        elif instr[0] == "SUB.D":
            dst = instr[1]
            src1 = instr[2]
            src2 = instr[3]
            res.append(SUBD(dst, src1, src2))
        elif instr[0] == "DIV.D":
            dst = instr[1]
            src1 = instr[2]
            src2 = instr[3]
            res.append(DIVD(dst, src1, src2))
    
    return res

instrs = [
    ("L.D", FloatRegister.F6, 34, Register.R2),
    ("L.D", FloatRegister.F2, 45, Register.R3),
    ("MUL.D", FloatRegister.F0, FloatRegister.F2, FloatRegister.F4),
    ("SUB.D", FloatRegister.F8, FloatRegister.F2, FloatRegister.F6),
    ("DIV.D", FloatRegister.F10, FloatRegister.F0, FloatRegister.F6),
    ("ADD.D", FloatRegister.F6, FloatRegister.F8, FloatRegister.F2)
]

instrs: list[Instruction] = parser(instrs)
for instr in instrs:
    instruction_status[instr] = InstructionStatus()

instruction_status.items()

In [6]:

def wb(cycle):
    for (fu, fu_entry) in functional_unit_status.table.items():
        if not fu_entry.instr:
            continue
        if not instruction_status[fu_entry.instr].is_exec():
            continue
        if fu_entry.time != 0:
            continue
        
        flag = True
        for (f, f_entry) in functional_unit_status.table.items():
            if not ((f_entry.s1 != fu_entry.dst or f_entry.s1_ready == False) and (f_entry.s2 != fu_entry.dst or f_entry.s2_ready == False)):
                flag = False
        if not flag:
            continue
        instruction_status[fu_entry.instr].wb = cycle
        for (f, f_entry) in functional_unit_status.table.items():
            if f_entry.s1_fu == fu:
                f_entry.s1_ready = True
            if f_entry.s2_fu == fu:
                f_entry.s2_ready = True
        
        register_result_status.table[fu_entry.dst] = None
        fu_entry.busy = False
        
        fu_entry.clear()
        
def exec(cycle):
    for (fu, fu_entry) in functional_unit_status.table.items():
        
        if not fu_entry.instr:
            continue
        if not instruction_status[fu_entry.instr].is_exec():
            continue
        
        if fu_entry.time >= 0:
           fu_entry.time -= 1
           
        if fu_entry.time == 0:
            instruction_status[fu_entry.instr].exec_comp = cycle
        
def read_oprands(cycle):
    for (fu, fu_entry) in functional_unit_status.table.items():
        
        if not fu_entry.instr:
            continue
        if not instruction_status[fu_entry.instr].is_read_op():
            continue
        
        if not (fu_entry.s1_ready and fu_entry.s2_ready):
            continue
        
        fu_entry.s1_ready, fu_entry.s2_ready = False, False
        fu_entry.s1_fu, fu_entry.s2_fu = None, None
        
        instruction_status[fu_entry.instr].read_op = cycle
        fu_entry.time = fu_entry.instr.delay
        
def issue(instr, cycle) -> bool:
    is_issue = False
    if type(instr) == LD:
        if functional_unit_status.table[FunctionalUnit.Integer] and (not register_result_status.table[instr.dst]):
            functional_unit_status.table[FunctionalUnit.Integer].busy = True
            functional_unit_status.table[FunctionalUnit.Integer].op = "load"
            functional_unit_status.table[FunctionalUnit.Integer].dst = instr.dst
            functional_unit_status.table[FunctionalUnit.Integer].s2 = instr.addr
            functional_unit_status.table[FunctionalUnit.Integer].s2_fu = register_result_status.table[instr.addr]
            functional_unit_status.table[FunctionalUnit.Integer].s2_ready = not functional_unit_status.table[FunctionalUnit.Integer].s2_fu
            register_result_status.table[instr.dst] = FunctionalUnit.Integer
            is_issue = True
        
    elif type(instr) == MULD:
        if functional_unit_status.table[FunctionalUnit.Mult1] and (not register_result_status.table[instr.dst]):
            functional_unit_status.table[FunctionalUnit.Mult1].busy = True
            functional_unit_status.table[FunctionalUnit.Mult1].op = "mult"
            functional_unit_status.table[FunctionalUnit.Mult1].dst = instr.dst
            functional_unit_status.table[FunctionalUnit.Mult1].s1 = instr.src1
            functional_unit_status.table[FunctionalUnit.Mult1].s1_fu = register_result_status.table[instr.src1]
            functional_unit_status.table[FunctionalUnit.Mult1].s1_ready = not functional_unit_status.table[FunctionalUnit.Mult1].s1_fu
            functional_unit_status.table[FunctionalUnit.Mult1].s2 = instr.src2
            functional_unit_status.table[FunctionalUnit.Mult1].s2_fu = register_result_status.table[instr.src2]
            functional_unit_status.table[FunctionalUnit.Mult1].s2_ready = not functional_unit_status.table[FunctionalUnit.Mult1].s2_fu
            register_result_status.table[instr.dst] = FunctionalUnit.Mult1
            is_issue = True
        elif functional_unit_status.table[FunctionalUnit.Mult2] and (not register_result_status.table[instr.dst]):
            functional_unit_status.table[FunctionalUnit.Mult2].busy = True
            functional_unit_status.table[FunctionalUnit.Mult2].op = "mult"
            functional_unit_status.table[FunctionalUnit.Mult2].dst = instr.dst
            functional_unit_status.table[FunctionalUnit.Mult2].s1 = instr.src1
            functional_unit_status.table[FunctionalUnit.Mult2].s1_fu = register_result_status.table[instr.src1]
            functional_unit_status.table[FunctionalUnit.Mult2].s1_ready = not functional_unit_status.table[FunctionalUnit.Mult2].s1_fu
            functional_unit_status.table[FunctionalUnit.Mult2].s2 = instr.src2
            functional_unit_status.table[FunctionalUnit.Mult2].s2_fu = register_result_status.table[instr.src2]
            functional_unit_status.table[FunctionalUnit.Mult2].s2_ready = not functional_unit_status.table[FunctionalUnit.Mult2].s2_fu
            register_result_status.table[instr.dst] = FunctionalUnit.Mult2
            is_issue = True
    
    elif type(instr) == SUBD:
        if functional_unit_status.table[FunctionalUnit.Add] and (not register_result_status.table[instr.dst]):
            functional_unit_status.table[FunctionalUnit.Add].busy = True
            functional_unit_status.table[FunctionalUnit.Add].op = "sub"
            functional_unit_status.table[FunctionalUnit.Add].dst = instr.dst
            functional_unit_status.table[FunctionalUnit.Add].s1 = instr.src1
            functional_unit_status.table[FunctionalUnit.Add].s1_fu = register_result_status.table[instr.src1]
            functional_unit_status.table[FunctionalUnit.Add].s1_ready = not functional_unit_status.table[FunctionalUnit.Add].s1_fu
            functional_unit_status.table[FunctionalUnit.Add].s2 = instr.src2
            functional_unit_status.table[FunctionalUnit.Add].s2_fu = register_result_status.table[instr.src2]
            functional_unit_status.table[FunctionalUnit.Add].s2_ready = not functional_unit_status.table[FunctionalUnit.Add].s2_fu
            is_issue = True
    
    elif type(instr) == ADDD:
        if functional_unit_status.table[FunctionalUnit.Add] and (not register_result_status.table[instr.dst]):
            functional_unit_status.table[FunctionalUnit.Add].busy = True
            functional_unit_status.table[FunctionalUnit.Add].op = "add"
            functional_unit_status.table[FunctionalUnit.Add].dst = instr.dst
            functional_unit_status.table[FunctionalUnit.Add].s1 = instr.src1
            functional_unit_status.table[FunctionalUnit.Add].s1_fu = register_result_status.table[instr.src1]
            functional_unit_status.table[FunctionalUnit.Add].s1_ready = not functional_unit_status.table[FunctionalUnit.Add].s1_fu
            functional_unit_status.table[FunctionalUnit.Add].s2 = instr.src2
            functional_unit_status.table[FunctionalUnit.Add].s2_fu = register_result_status.table[instr.src2]
            functional_unit_status.table[FunctionalUnit.Add].s2_ready = not functional_unit_status.table[FunctionalUnit.Add].s2_fu
            is_issue = True
            
    elif type(instr) == DIVD:
        if functional_unit_status.table[FunctionalUnit.Divide] and (not register_result_status.table[instr.dst]):
            functional_unit_status.table[FunctionalUnit.Divide].busy = True
            functional_unit_status.table[FunctionalUnit.Divide].op = "add"
            functional_unit_status.table[FunctionalUnit.Divide].dst = instr.dst
            functional_unit_status.table[FunctionalUnit.Divide].s1 = instr.src1
            functional_unit_status.table[FunctionalUnit.Divide].s1_fu = register_result_status.table[instr.src1]
            functional_unit_status.table[FunctionalUnit.Divide].s1_ready = not functional_unit_status.table[FunctionalUnit.Divide].s1_fu
            functional_unit_status.table[FunctionalUnit.Divide].s2 = instr.src2
            functional_unit_status.table[FunctionalUnit.Divide].s2_fu = register_result_status.table[instr.src2]
            functional_unit_status.table[FunctionalUnit.Divide].s2_ready = not functional_unit_status.table[FunctionalUnit.Divide].s2_fu
            is_issue = True
    
    if is_issue:
        instruction_status[instr].issue = cycle
            
        
        
        

        